# Field Aliases and Default Values

So far we have seen how to define field names, types and default validation behavior using just type hinting and default values.

But Pydantic has a more powerful way of attaching field configuration to fields in our models.

This is done via the `Field` object.

To configure fields beyond what we've seen so far, we can create an instance of this `Field` object, configure settings on it, and attach it to a field by using it as the default value.

This means we'll need a slightly different way to specify both a `Field` and a default value at the same time (we'll specify the default value in the `Field` object itself).

Let's take a look at how we can use the `Field` object to define aliases for our fields.

In [1]:
from pydantic import BaseModel, Field, ValidationError

In [2]:
class Model(BaseModel):
    id_: int = Field(alias="id")
    last_name: str = Field(alias="lastName")

Now we can deserialize data that uses the aliases, but get mapped to our more Pythonic field names.

In [3]:
json_data = """
{
    "id": 100,
    "lastName": "Gauss"
}
"""

m = Model.model_validate_json(json_data)

In [4]:
m

Model(id_=100, last_name='Gauss')

As you can see the model fields have been correctly populated.

So the data being deserialized **must** use the aliases, not the field names, including when we create an instance of a model using keyword arguments, not just when deserializing a dictionary or JSON.

In [5]:
Model(id=100, lastName="Gauss")

Model(id_=100, last_name='Gauss')

In [6]:
try:
    Model(id_=100, last_name="Gauss")
except ValidationError as ex:
    print(ex)

2 validation errors for Model
id
  Field required [type=missing, input_value={'id_': 100, 'last_name': 'Gauss'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
lastName
  Field required [type=missing, input_value={'id_': 100, 'last_name': 'Gauss'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


How about accessing the fields on the instance? alias or field name?

Well, our model is a regular Python class, and the class attributes are the field names, not the aliases.

So, we need to use the field names:

In [7]:
m.last_name

'Gauss'

And indeed, `lastName` is **not** an attribute of the instance:

In [8]:
hasattr(m, "lastName")

False

In [9]:
dict(m)

{'id_': 100, 'last_name': 'Gauss'}

What if we also want to define a default for these fields? We can no longer use our previous approach - instead we can add the default to the `Field` object as follows:

In [10]:
class Model(BaseModel):
    id_: int = Field(alias="id", default=100)
    last_name: str = Field(alias="lastName")

In [11]:
Model(lastName="Gauss")

Model(id_=100, last_name='Gauss')

In [12]:
Model(id=1, lastName="Newton")

Model(id_=1, last_name='Newton')

When we serialize a model, the field names, not the aliases, will be used:

In [13]:
m.model_dump()

{'id_': 100, 'last_name': 'Gauss'}

In other words, when we define an alias, the default behavior for Pydantic is that they are used when **deserializing** ONLY.

The representation uses the field names, the attributes on the class are the field names, and serialized data used the field names.

We'll see later how we can configure our model to allow deserializing using either the alias or the field name.

We can also indicate how we want the model to be serialized - the default is field names, but we can specifically request Pydantic to use the aliases(if present) instead.

Let's take a look:

In [14]:
class Person(BaseModel):
    id_: int = Field(alias="id")
    first_name: str | None = Field(alias="firstName", default=None)
    last_name: str = Field(alias="lastName")
    age: int | None = None

As you can see all fields except `age` have an alias defined.

Let's create a model instance:

In [15]:
isaac = Person(id=1, firstName="Isaac", lastName="Newton", age=84)
isaac

Person(id_=1, first_name='Isaac', last_name='Newton', age=84)

In [16]:
isaac.model_dump()

{'id_': 1, 'first_name': 'Isaac', 'last_name': 'Newton', 'age': 84}

The above serialization use the field names. To use the aliases instead (or the field name if no alias is defined), we supply an argument to the dump methods:

In [17]:
isaac.model_dump(by_alias=True)

{'id': 1, 'firstName': 'Isaac', 'lastName': 'Newton', 'age': 84}

In [18]:
isaac.model_dump_json(by_alias=True)

'{"id":1,"firstName":"Isaac","lastName":"Newton","age":84}'

Let's look at the model field info so we can see how that is now getting more complex as we add more configuration to our models and fields:

In [19]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, alias='firstName', alias_priority=2),
 'last_name': FieldInfo(annotation=str, required=True, alias='lastName', alias_priority=2),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False)}

You'll notice something in the properties: `alias_priority=2`.

Technically Pydantic offers three configurations for aliases (I only mentioned two in the slides):
- alias
- validation alias
- serialization alias

  When you use all three, you can also indicate which alias takes precedence when deserializing and serializing. I never use this feature, and could not come up with a good reason to do so. So, I'm not going to cover it in this course - I find it rather confusing, and not something I see as commonly needed and beyond the scope of an essentials course.